# FZ with OpenModelica

This notebook demonstrates using FZ with OpenModelica for parametric simulations of dynamic systems.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Funz/fz.github.io/blob/main/notebooks/modelica_example.ipynb)

## Install OpenModelica and FZ

In [ ]:
# Install OpenModelica
!apt-get update -qq
!apt-get install -y omc > /dev/null 2>&1

# Install FZ
!pip install git+https://github.com/Funz/fz.git -q

print("✓ Installation complete")

## Create Modelica Model

We'll create a simple harmonic oscillator with parametric damping and frequency:

In [ ]:
%%writefile Oscillator.mo
model Oscillator
  parameter Real omega = $omega;  // Natural frequency (rad/s)
  parameter Real zeta = $zeta;    // Damping ratio
  
  Real x(start=1.0);  // Position
  Real v(start=0.0);  // Velocity
  
equation
  der(x) = v;
  der(v) = -omega^2 * x - 2*zeta*omega*v;
end Oscillator;

## Create Simulation Script

This script compiles and runs the Modelica model:

In [ ]:
%%writefile simulate.sh
#!/bin/bash

# Source input to get parameters
source $1

# Compile and simulate Modelica model
omc << EOF
loadFile("Oscillator.mo");
buildModel(Oscillator);
quit();
EOF

# Run simulation with parameters
./Oscillator -override omega=$omega,zeta=$zeta > /dev/null 2>&1

# Extract peak overshoot from results
python3 << PYEOF
import csv
import os

# Read CSV results
if os.path.exists('Oscillator_res.csv'):
    with open('Oscillator_res.csv', 'r') as f:
        reader = csv.reader(f)
        data = list(reader)
        # Find position column (x)
        header = data[0]
        x_idx = header.index('x')
        x_values = [float(row[x_idx]) for row in data[1:] if row[x_idx]]
        peak = max(abs(x) for x in x_values)
        print(f"peak_overshoot = {peak}")
else:
    print("peak_overshoot = 0")
PYEOF

echo "Simulation complete"

In [ ]:
!chmod +x simulate.sh

## Create Input Template

In [ ]:
%%writefile input.txt
omega=$omega
zeta=$zeta

## Run Parametric Study

Study how damping ratio and frequency affect the system response:

In [ ]:
import fz

# Define FZ model
model = {
    "varprefix": "$",
    "output": {
        "peak_overshoot": "grep 'peak_overshoot = ' *.txt 2>/dev/null | awk '{print $3}' || echo 0"
    }
}

# Parameter ranges
input_variables = {
    "omega": [1, 2, 5, 10],          # 4 frequencies
    "zeta": [0.1, 0.3, 0.5, 0.7, 1.0] # 5 damping ratios
}

# Run parametric study
results = fz.fzr(
    "input.txt",
    input_variables,
    model,
    calculators="sh://bash simulate.sh",
    results_dir="oscillator_results"
)

print(f"\nCompleted {len(results)} simulations")
results

## Visualize Results

Create a heatmap showing peak overshoot for different damping and frequency values:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create pivot table
pivot = results.pivot(index='zeta', columns='omega', values='peak_overshoot')

plt.figure(figsize=(10, 6))
sns.heatmap(pivot, annot=True, fmt='.2f', cmap='RdYlGn_r', cbar_kws={'label': 'Peak Overshoot'})
plt.title('Oscillator Response: Peak Overshoot vs Damping and Frequency')
plt.xlabel('Natural Frequency ω (rad/s)')
plt.ylabel('Damping Ratio ζ')
plt.tight_layout()
plt.show()

## Analysis

Analyze the relationship between damping and overshoot:

In [ ]:
# Plot overshoot vs damping for each frequency
plt.figure(figsize=(10, 6))

for omega in sorted(results['omega'].unique()):
    data = results[results['omega'] == omega].sort_values('zeta')
    plt.plot(data['zeta'], data['peak_overshoot'], 
             marker='o', label=f'ω = {omega} rad/s')

plt.xlabel('Damping Ratio ζ')
plt.ylabel('Peak Overshoot')
plt.title('Effect of Damping on Peak Overshoot')
plt.legend()
plt.grid(True)
plt.show()

# Find optimal damping (minimum overshoot)
for omega in sorted(results['omega'].unique()):
    data = results[results['omega'] == omega]
    best = data.loc[data['peak_overshoot'].idxmin()]
    print(f"ω={omega}: Optimal ζ={best['zeta']}, Peak={best['peak_overshoot']:.3f}")

## Next Steps

Try these extensions:

1. **Different initial conditions**: Vary `x(start)` and `v(start)`
2. **Forced oscillations**: Add external forcing term
3. **Nonlinear systems**: Add nonlinear damping or stiffness
4. **Multi-DOF systems**: Extend to coupled oscillators

## Learn More

- [FZ Documentation](https://funz.github.io)
- [OpenModelica](https://openmodelica.org/)
- [More Examples](https://funz.github.io/examples/)